In [7]:
import pandas as pd
import plotly.express as px

# --- CONFIGURATION ---
geojson_url = "https://raw.githubusercontent.com/holtzy/D3-graph-gallery/master/DATA/world.geojson"

# Ensure these filenames match your uploads exactly
file_co2 = "co-emissions-per-capita.csv"
file_hdi = "human-development-index.csv"
file_temp = "monthly-temperature-anomalies.csv"

# Define column names for plotting for better readability and maintainability
co2_col = "emissions_total_per_capita"
hdi_col = "hdi__sex_total" # Corrected column name as per error message and dataframe inspection
temp_col = "temperature_anomaly"

# --- 1. LOAD DATA ---
print("Loading datasets...")
df_co2 = pd.read_csv(file_co2)
df_hdi = pd.read_csv(file_hdi)
df_temp = pd.read_csv(file_temp)

# --- 2. PRE-PROCESS TEMPERATURE (Monthly -> Yearly) ---
print("Processing Temperature Data...")
# Convert 'Day' column to datetime objects
df_temp["Date"] = pd.to_datetime(df_temp["Day"])
# Extract just the Year
df_temp["Year"] = df_temp["Date"].dt.year
# Group by Country and Year to get the Annual Average
# We use 'temperature_anomaly' based on your screenshot
df_temp_yearly = df_temp.groupby(['Entity', 'Code', 'Year'])[temp_col].mean().reset_index()

# --- 3. FILTER ALL DATA (2010 - 2023) ---
def filter_years(df):
    return df[(df["Year"] >= 2010) & (df["Year"] <= 2023)].sort_values("Year")

df_co2 = filter_years(df_co2)
df_hdi = filter_years(df_hdi)
df_temp_yearly = filter_years(df_temp_yearly)

# --- 4. GENERATE MAPS ---

# Map 1: CO2 Emissions
print("Generating CO2 Map...")
fig_co2 = px.choropleth(
    df_co2,
    geojson=geojson_url,
    locations="Code",
    featureidkey="id",
    color=co2_col, # Using the defined variable
    animation_frame="Year",
    color_continuous_scale="Reds",
    range_color=(0, 20),
    title="CO2 Emissions per Capita (2010-2023)",
    projection="natural earth",
    hover_name="Entity"
)
fig_co2.update_geos(fitbounds="locations", visible=True)
fig_co2.update_layout(margin={"r":0,"t":50,"l":0,"b":0})


# Map 2: Human Development Index
print("Generating HDI Map...")
fig_hdi = px.choropleth(
    df_hdi,
    geojson=geojson_url,
    locations="Code",
    featureidkey="id",
    color=hdi_col, # Using the defined variable
    animation_frame="Year",
    color_continuous_scale="Viridis",
    range_color=(0.4, 1.0), # Standard HDI range
    title="Human Development Index (2010-2023)",
    projection="natural earth",
    hover_name="Entity"
)
fig_hdi.update_geos(fitbounds="locations", visible=True)
fig_hdi.update_layout(margin={"r":0,"t":50,"l":0,"b":0})


# Map 3: Temperature Anomalies
print("Generating Temperature Map...")
fig_temp = px.choropleth(
    df_temp_yearly,
    geojson=geojson_url,
    locations="Code",
    featureidkey="id",
    color=temp_col, # Using the defined variable
    animation_frame="Year",
    color_continuous_scale="RdBu_r", # Red-Blue Reverse (Red = Hot)
    range_color=(-1.0, 2.5),         # Range based on typical anomalies
    title="Average Annual Temperature Anomaly (2010-2023)",
    projection="natural earth",
    hover_name="Entity"
)
fig_temp.update_geos(fitbounds="locations", visible=True)
fig_temp.update_layout(margin={"r":0,"t":50,"l":0,"b":0})

# --- 5. DISPLAY ---
fig_co2.show()
fig_hdi.show()
fig_temp.show()

Loading datasets...
Processing Temperature Data...
Generating CO2 Map...
Generating HDI Map...
Generating Temperature Map...
